In [1]:
import openai

In [2]:
client = openai.Client()

In [3]:
vector_store = client.beta.vector_stores.create(
    name="Tutor de Apostilas"
)

In [4]:
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [5]:
file_batch.status

'completed'

In [6]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [7]:
assistant = client.beta.assistants.create(
    name="Tutor Apostila",
    instructions="Você é um tutor especializado em tecnologias emergentes. \
        Você sabe responder perguntas sobre LLMs como OpenAI, HuggingFace, etc. \
            Caso você não encontre as resostas, seja sincero e fale que não sabe responder",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids":[vector_store.id]}},
    model="gpt-4-turbo-preview"
)

In [14]:
pergunta = "Conforme o documento, o que é o OpenAI?"

In [15]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [16]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [17]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [18]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_AclquuDVuyvFclArD4RlUcfK', assistant_id='asst_WXJuetM32uqZclf5n95Far6C', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=916, file_citation=FileCitation(file_id='file-HZGXb8BPup3LEiY4fJbxXf'), start_index=903, text='【4:4†LLM.pdf】', type='file_citation')], value='A OpenAI, conforme descrito no documento, é uma organização que foi fundamental no desenvolvimento e lançamento de modelos de linguagem avançados, como o GPT-3 e o ChatGPT. Em 2020, a OpenAI lançou o GPT-3, o maior modelo naquela época, com 175 bilhões de parâmetros, estabelecendo novos padrões de desempenho para tarefas relacionadas à linguagem. Posteriormente, em 2022, o ChatGPT foi introduzido, transformando o GPT-3 e modelos semelhantes num serviço largamente acessível aos usuários por meio de uma interface web, o que impulsionou significativamente a conscientização pública sobre os modelos de linguagem de gran

In [19]:
print(mensagens.data[0].content[0].text.value)

A OpenAI, conforme descrito no documento, é uma organização que foi fundamental no desenvolvimento e lançamento de modelos de linguagem avançados, como o GPT-3 e o ChatGPT. Em 2020, a OpenAI lançou o GPT-3, o maior modelo naquela época, com 175 bilhões de parâmetros, estabelecendo novos padrões de desempenho para tarefas relacionadas à linguagem. Posteriormente, em 2022, o ChatGPT foi introduzido, transformando o GPT-3 e modelos semelhantes num serviço largamente acessível aos usuários por meio de uma interface web, o que impulsionou significativamente a conscientização pública sobre os modelos de linguagem de grande escala (LLMs) e IA generativa. O documento também menciona o lançamento de modelos de linguagem de código aberto que têm mostrado resultados impressionantes, bem como o lançamento do GPT-4 em 2023, que estabeleceu novos benchmarks em termos de tamanho de parâmetros e desempenho【4:4†LLM.pdf】.
